In [111]:
import os
from time import sleep

import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementNotInteractableException
from selenium.common.exceptions import StaleElementReferenceException
from bs4 import BeautifulSoup

In [112]:
p_name = []
p_address = []
p_rating = []
p_review = []
p_tag = []

In [113]:
def main():
    global driver, load_wb, review_num

    driver.get('https://map.kakao.com/')  # 주소 가져오기

    # 검색할 목록
    place_infos = ['울산 명소', '울산 볼링장', '울산 공원', '울산 만화카페', '울산 멀티방', '울산 방탈출', '울산 보드카페', '울산 코인노래방', '울산 숙소']

    for i, place in enumerate(place_infos):
        search(place)

    driver.quit()
    print("finish")


def search(place):
    global driver

    search_area = driver.find_element(By.XPATH, '//*[@id="search.keyword.query"]')  # 검색 창
    search_area.send_keys(place)  # 검색어 입력
    driver.find_element(By.XPATH, '//*[@id="search.keyword.submit"]').send_keys(Keys.ENTER)  # Enter로 검색
    driver.find_element(By.XPATH, '//*[@id="info.main.options"]/li[1]/a').send_keys(Keys.ENTER)

    # 검색된 정보가 있는 경우에만 탐색
    # 1번 페이지 place list 읽기
    html = driver.page_source

    soup = BeautifulSoup(html, 'html.parser')
    place_lists = soup.select('.placelist > .PlaceItem') # 검색된 장소 목록

    # 검색된 첫 페이지 장소 목록 크롤링하기
    crawling(place, place_lists)
    search_area.clear()

    # 우선 더보기 클릭해서 2페이지
    try:
        driver.find_element(By.XPATH, '//*[@id="info.search.place.more"]').send_keys(Keys.ENTER)

        # 2~ 5페이지 읽기
        for i in range(2, 6):
            # 페이지 넘기기
            xPath = '//*[@id="info.search.page.no' + str(i) + '"]'
            if i == 5:
                xPath = '//*[@id="info.search.page.next"]'
            driver.find_element(By.XPATH, xPath).send_keys(Keys.ENTER)

            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')
            place_lists = soup.select('.placelist > .PlaceItem') # 장소 목록 list

            crawling(place, place_lists)
        
        for i in range(1,7):
          if i == 6:
            for i in range(3, 5):
              # 페이지 넘기기
              xPath = '//*[@id="info.search.page.no' + str(i) + '"]'
              driver.find_element(By.XPATH, xPath).send_keys(Keys.ENTER)

              html = driver.page_source
              soup = BeautifulSoup(html, 'html.parser')
              place_lists = soup.select('.placelist > .PlaceItem') # 장소 목록 list

              crawling(place, place_lists)
          else:
            for i in range(3, 7):
              # 페이지 넘기기
              xPath = '//*[@id="info.search.page.no' + str(i) + '"]'
              if i == 6:
                  xPath = '//*[@id="info.search.page.next"]'
              driver.find_element(By.XPATH, xPath).send_keys(Keys.ENTER)

              html = driver.page_source
              soup = BeautifulSoup(html, 'html.parser')
              place_lists = soup.select('.placelist > .PlaceItem') # 장소 목록 list

              crawling(place, place_lists)

    except ElementNotInteractableException:
        print('not found')
    finally:
        search_area.clear()


def crawling(place_key, place_lists):

    while_flag = False
    for i, place in enumerate(place_lists):

        place_name = place.select('.head_item > .tit_name > .link_name')[0].text  # place name
        p_name.append(place_name)
        place_address = place.select('.info_item > .addr > p')[0].text  # place address
        p_address.append(place_address)
        place_rating = place.select('.rating > .score > em')[0].text
        p_rating.append(place_rating)
        

        detail_page_xpath = '//*[@id="info.search.place.list"]/li[' + str(i + 1) + ']/div[5]/div[4]/a[1]'
        driver.find_element(By.XPATH, detail_page_xpath).send_keys(Keys.ENTER)
        driver.switch_to.window(driver.window_handles[-1])  # 상세정보 탭으로 변환

        p_review.append(extract_review(place_key, place_name))
        p_tag.append(extract_tag(place_name))

        driver.close()
        driver.switch_to.window(driver.window_handles[0])  # 검색 탭으로 전환


def extract_review(place_key, place_id):
    review_df=pd.DataFrame(columns={'p_id','u_id','u_rate','review'})
    
    while True:
        try:
            if driver.find_element(By.CSS_SELECTOR, '#mArticle > div.cont_evaluation > div.evaluation_review > a').text == '후기 더보기':
                driver.find_element(By.CSS_SELECTOR, '#mArticle > div.cont_evaluation > div.evaluation_review > a').click()
            else:
                break
        except:
            break
        sleep(0.5)
    
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    try:
        review_lists = soup.find('ul', {'class':'list_evaluation'}).find_all('li')
    except:
        return ''
    
    p_name = []
    u_name = []
    u_rating = []
    u_comment = []
    
    for review_list in review_lists:
        try:
            user_id = review_list.find('a', {'class':'link_user'}).text
        except:
            user_id = ''
            continue
        
        try:
            value = review_list.find('div', {'class':'star_info'}).find('div').find('span').find('span')['style']
            value = value.replace('width:', '')
            value = value.replace('%;', '')
            user_rating = int(value)/20
        except:
            user_rating = ''
            continue
        
        try:
            user_comment = review_list.find('p', {'class':'txt_comment'}).find('span').text
        except:
            user_comment = ''
            continue
        p_name.append(place_id)
        u_name.append(user_id)
        u_rating.append(user_rating)
        u_comment.append(user_comment)
        
        print('place_id:', place_id)
        print('user_id:', user_id)
        print('score:', user_rating)
        print('comment:', user_comment)
    
    review_df = pd.DataFrame({
        'place_id':p_name,
        'user_id':u_name,
        'score':u_rating,
        'comment':u_comment
    },)

    review_df.to_csv(f'./kakao_reviews/{place_key}/{place_id}.csv', index = False, header=True, encoding='utf-8-sig')
    return f'./kakao_reviews/{place_key}/{place_id}.csv'

def extract_tag(place_name):
    global driver

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    # 첫 페이지 리뷰 목록 찾기
    tag_lists = soup.select('.tag_g > a')

    tags = []
    # 리뷰가 있는 경우
    if len(tag_lists) != 0:
        for i, tag in enumerate(tag_lists):
            if len(tag) != 0:
                tags.append(tag.text)

    return tags

In [114]:
options = webdriver.ChromeOptions()
options.add_argument('lang=ko_KR')
chromedriver_path = "chromedriver"
driver = webdriver.Chrome(os.path.join(os.getcwd(), chromedriver_path), options=options)  # chromedriver 열기
driver.implicitly_wait(10)

C:\Users\82107\AppData\Local\Temp\ipykernel_36520\2146430641.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(os.path.join(os.getcwd(), chromedriver_path), options=options)  # chromedriver 열기


In [115]:
main()

place_id: 일산해수욕장
user_id: 떡꼬치
score: 5.0
comment: 이뻐요 낭만 있슘당 그렇게 사람 많지도 않고 사진 찍기 좋아요.
place_id: 일산해수욕장
user_id: 웽
score: 3.0
comment: 그냥그럼
place_id: 일산해수욕장
user_id: flow
score: 5.0
comment: 
place_id: 일산해수욕장
user_id: 김수완
score: 5.0
comment: 
place_id: 일산해수욕장
user_id: ^^ㄲ ㅐㅂ ㅣ
score: 4.0
comment: 
place_id: 일산해수욕장
user_id: 압둘라
score: 5.0
comment: 
place_id: 일산해수욕장
user_id: 쯧쯧
score: 5.0
comment: 여기가 모래사장에 개인풀장 설치하고 노는 돌게이가 다녀간 곳인가요??
place_id: 일산해수욕장
user_id: NakiT
score: 3.0
comment: 
place_id: 일산해수욕장
user_id: ㅁㅁ
score: 1.0
comment: 
place_id: 일산해수욕장
user_id: 호이챵이
score: 4.0
comment: 
place_id: 일산해수욕장
user_id: 지니팡
score: 4.0
comment: 코로나때문에 폐장이 되어서 들여다보지는 못했지만 울산에서 가장 큰 해수욕장인가봐요볼거리도 많고 해양레포츠도 즐길수있어요다만 펜션보다는 모텔시설이 많은듯해요
place_id: 일산해수욕장
user_id: Serna
score: 4.0
comment: A great quiet beach in a great location with lots to see, eat and drink (cafes)! Right next to the famous light house and park! They have a great water park nearby!
place_id: 일산해수욕장
user_id: .
score: 2.0
comment: 물밑고기들이

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=110.0.5481.178)
Stacktrace:
Backtrace:
	(No symbol) [0x004B37D3]
	(No symbol) [0x00448B81]
	(No symbol) [0x0034B36D]
	(No symbol) [0x0032E5DA]
	(No symbol) [0x0039F0CB]
	(No symbol) [0x003B0A56]
	(No symbol) [0x0039B216]
	(No symbol) [0x00370D97]
	(No symbol) [0x0037253D]
	GetHandleVerifier [0x0072ABF2+2510930]
	GetHandleVerifier [0x00758EC1+2700065]
	GetHandleVerifier [0x0075C86C+2714828]
	GetHandleVerifier [0x00563480+645344]
	(No symbol) [0x00450FD2]
	(No symbol) [0x00456C68]
	(No symbol) [0x00456D4B]
	(No symbol) [0x00460D6B]
	BaseThreadInitThunk [0x768F7D69+25]
	RtlInitializeExceptionChain [0x77D8BB9B+107]
	RtlClearBits [0x77D8BB1F+191]


In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame({
  'p_name':p_name,
  'p_address':p_address,
  'p_rating':p_rating,
  'p_tag':p_tag
},)
df.to_csv('kakaoMap.csv',encoding='utf-8-sig')